Script used for data augmentation - generating new images; new image for each of the original images

It uses several transformation functions:

* random_rotation(image): Rotates the image by a random angle between -30 and 30 degrees.
* random_translation(image): Shifts the image randomly in both x and y directions by up to 20% of its dimensions.
* random_flip(image): Horizontally flips the image with a 50% probability.
* random_zoom(image): Zooms in or out on the image by a factor between 0.8 and 1.2, cropping or padding as needed.
* random_brightness_contrast(image): Randomly adjusts the brightness and contrast of the image.
* add_noise(image): Adds random Gaussian noise to the image.
* random_perspective(image): Applies a random perspective transformation to the image.
* augment_image(image): Applies a random combination of the above transformations to an image.
* augment_images_in_subfolders(base_folder): Processes all images in a specified folder, applying augmentations and saving the new images with modified filenames.

In [ ]:
import cv2
import os
import random
import numpy as np

In [ ]:
def random_rotation(image):
    angle = random.uniform(-30, 30)
    h, w = image.shape[:2]
    M = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
    return cv2.warpAffine(image, M, (w, h))

In [ ]:
def random_translation(image):
    h, w = image.shape[:2]
    tx = random.uniform(-0.2, 0.2) * w
    ty = random.uniform(-0.2, 0.2) * h
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    return cv2.warpAffine(image, M, (w, h))

In [ ]:
def random_flip(image):
    if random.random() < 0.5:
        return cv2.flip(image, 1)
    return image

In [ ]:
def random_zoom(image):
    h, w = image.shape[:2]
    scale = random.uniform(0.8, 1.2)
    new_h, new_w = int(h * scale), int(w * scale)
    image = cv2.resize(image, (new_w, new_h))
    if scale > 1.0:
        start_x = (new_w - w) // 2
        start_y = (new_h - h) // 2
        return image[start_y:start_y + h, start_x:start_x + w]
    else:
        pad_x = (w - new_w) // 2
        pad_y = (h - new_h) // 2
        return cv2.copyMakeBorder(image, pad_y, pad_y, pad_x, pad_x, cv2.BORDER_CONSTANT, value=[0, 0, 0])

In [ ]:
def random_brightness_contrast(image):
    alpha = random.uniform(0.8, 1.2)  # Contrast control
    beta = random.randint(-30, 30)    # Brightness control
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

In [ ]:
def add_noise(image):
    row, col, ch = image.shape
    mean = 0
    var = 10
    sigma = var ** 0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    noisy = image + gauss.reshape(row, col, ch)
    return np.clip(noisy, 0, 255).astype(np.uint8)

In [ ]:
def random_perspective(image):
    h, w = image.shape[:2]
    pts1 = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
    shift = 50
    pts2 = np.float32([[random.randint(-shift, shift), random.randint(-shift, shift)],
                       [w + random.randint(-shift, shift), random.randint(-shift, shift)],
                       [random.randint(-shift, shift), h + random.randint(-shift, shift)],
                       [w + random.randint(-shift, shift), h + random.randint(-shift, shift)]])
    M = cv2.getPerspectiveTransform(pts1, pts2)
    return cv2.warpPerspective(image, M, (w, h))

In [ ]:
def augment_image(image):
    if random.random() < 0.5:
        image = random_rotation(image)
    if random.random() < 0.5:
        image = random_translation(image)
    if random.random() < 0.5:
        image = random_flip(image)
    if random.random() < 0.5:
        image = random_zoom(image)
    if random.random() < 0.5:
        image = random_brightness_contrast(image)
    if random.random() < 0.5:
        image = add_noise(image)
    if random.random() < 0.5:
        image = random_perspective(image)
    return image

In [ ]:
def augment_images_in_subfolders(base_folder):
    for class_name in os.listdir(base_folder):
        class_folder = os.path.join(base_folder, class_name)
        if not os.path.isdir(class_folder):
            continue

        for image_file in os.listdir(class_folder):
            image_path = os.path.join(class_folder, image_file)
            if not os.path.isfile(image_path):
                continue

            image = cv2.imread(image_path)
            if image is None:
                print(f"Warning: Unable to read image {image_path}")
                continue

            augmented_image = augment_image(image)
            augmented_image_name = f'{os.path.splitext(image_file)[0]}_aug.jpg'
            augmented_image_path = os.path.join(class_folder, augmented_image_name)
            cv2.imwrite(augmented_image_path, augmented_image)

    print("Data augmentation completed and augmented images are saved.")


In [ ]:
base_folder = '../datasets/TrashNet_augmentedx2'
augment_images_in_subfolders(base_folder)